In [ ]:
from ANGL import GNN

import argparse
import json
import operator
import os
import torch
import numpy as np
import torch.optim as optim
import datetime

from torch_geometric.loader import DataLoader
from functools import reduce
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
from evaluate import Evaluator
from dataset_handcraft_pyg import PygGraphPropPredDataset

reg_criterion = torch.nn.HuberLoss(delta=5)
cls_criterion = torch.nn.BCEWithLogitsLoss()

def eval_mape(pred, target):
    mape = 0
    for i in range(len(pred)):
        mape += np.abs((pred[i]-target[i])/target[i])
    return mape/len(pred)

def eval_rmse(pred, target):
    rmse = 0
    for i in range(len(pred)):
        rmse += (pred[i]-target[i])**2
    rmse = np.sqrt(rmse/len(pred))
    return rmse
def eval_smape(pred, target):
    smape = 0
    for i in range(len(pred)):
        smape += np.abs(pred[i]-target[i])/((np.abs(pred[i])+np.abs(target[i]))/2)
    smape /= len(pred)
    return smape

def eval_r2(pred, target):
    son = 0
    mother = 0
    miu = np.mean(target)
    for i in range(len(pred)):
        son += (pred[i]-target[i])**2
        mother += (target[i]-miu)**2
    r2 = 1 - son/mother
    return r2

def eval(model, device, loader, evaluator):
    model.eval()
    y_true = []
    y_pred = []

    for _, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)
        with torch.no_grad():
            pred = model(batch)
        y_true.append(batch.y.view(pred.shape).detach().cpu())
        y_pred.append(pred.detach().cpu())

    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    # for i in range(len(y_pred)):
    #     y_pred[i] *= 1000
    #     y_true[i] *= 1000
    mape = eval_mape(y_pred, y_true)
    print("mape:{}".format(mape))
    rmse = eval_rmse(y_pred, y_true)
    print("mse:{}".format(rmse))
    smape = eval_smape(y_pred, y_true)
    print("smape:{}".format(smape))
    r2 = eval_r2(y_pred, y_true)
    print("r2:{}".format(r2))
    input_dict = {"y_true": y_true, "y_pred": y_pred}

    return evaluator.eval(input_dict), y_true, y_pred


def eval_norm(model, device, loader, evaluator, min, max):
    model.eval()
    y_true = []
    y_pred = []

    for _, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)
        with torch.no_grad():
            pred = model(batch)
        y_true.append(batch.y.view(pred.shape).detach().cpu())
        y_pred.append(pred.detach().cpu())

    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()

    for i in range(len(y_pred)):
        y_pred[i] /= 100
        y_pred[i] = (y_pred[i])*(max-min)+ min
        y_true[i] /= 100
        y_true[i] = (y_true[i])*(max-min)+ min
    mape = eval_mape(y_pred, y_true)
    print("mape:{}".format(mape))
    rmse = eval_rmse(y_pred, y_true)
    print("mse:{}".format(rmse))
    input_dict = {"y_true": y_true, "y_pred": y_pred}

    return evaluator.eval(input_dict), y_true, y_pred


In [26]:
parser = argparse.ArgumentParser(description='GCN, GCN-virtual, GIN, GIN-virtual')
parser.add_argument('--device', type=int, default=3,
                        help='which gpu to use if any (default: 0)')
parser.add_argument('--gnn', type=str, default='sage',
                        help='GNN gin, gin-virtual, or gcn, or gcn-virtual (default: gin-virtual), sage, pna, rgcn, gat')
parser.add_argument('--drop_ratio', type=float, default=0.5,
                        help='dropout ratio (default: 0.3)')
parser.add_argument('--num_layer', type=int, default=5,
                        help='number of GNN message passing layers (default: 5)')
parser.add_argument('--emb_dim', type=int, default=100,
                        help='dimensionality of hidden units in GNNs (default: 300)')
parser.add_argument('--batch_size', type=int, default=1,
                        help='input batch size for training (default: 32)')
parser.add_argument('--epochs', type=int, default=300,
                        help='number of epochs to train (default: 300)')
parser.add_argument('--num_workers', type=int, default=0,
                        help='number of workers (default: 0)')
parser.add_argument('--dataset', type=str, default="latncey_mnist_mix_nonuniform",
                        help='dataset name')
parser.add_argument('--graph_pooling', type=str, default="sum",
                        help='sum, mean, max, attention')
parser.add_argument('--save_dir', type=str, default="logs",
                        help='dir of saved models')
parser.add_argument('--feature', type=str, default="full",
                        help='full feature or simple feature')
parser.add_argument('--filename', type=str, default="train.tensor",
                        help='filename to output result (default: )')
parser.add_argument('--train_log', type=str, default="train",
                        help='log file')
parser.add_argument('--norm', type=str, default=1000,
                        help='normalization factor')
args = parser.parse_args(args=[])

device = torch.device("cuda:" + str(args.device)) if torch.cuda.is_available() else torch.device("cpu")


# test module: dataset scale
test_module = "dataset"
test_dataset = "latency_ap_cifar10dvs"
dataset = PygGraphPropPredDataset(test_dataset, root='./data')
evaluator = Evaluator(test_dataset)
if test_module == "dataset":
    model_name = "./saved_model/latency_dataset.pt"
    print('This will take some time.......')
    for i in range(len(dataset.data.x)):
        dataset.data.x[i][1:] = dataset.data.x[i][1:] * dataset.data.x[i][0] * 2
    dataset.data.y = dataset.data.y/1000

elif test_module == "scale":
    model_name = "./saved_model/latency_scale.pt"
    print('This will take some time.......')
    for i in range(len(dataset.data.x)):
        dataset.data.x[i][1:] = dataset.data.x[i][1:] * dataset.data.x[i][0] * 2
    max_y = torch.max(dataset.data.y)
    min_y = torch.min(dataset.data.y)
    dataset.data.y = (dataset.data.y - min_y) / (max_y - min_y)*200

split_idx = dataset.get_idx_split([0.8, 0.1, 0.1])

train_loader = DataLoader(dataset[split_idx["train"]], batch_size=args.batch_size, shuffle=True,
                              num_workers=args.num_workers)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=args.batch_size, shuffle=False,
                              num_workers=args.num_workers)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=args.batch_size, shuffle=False,
                             num_workers=args.num_workers)


model = GNN(num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, net_type=args.gnn, graph_pooling=args.graph_pooling).to(device)

model.load_state_dict(torch.load(model_name)['model_state_dict'])
# train_perf, _, _ = eval_norm(model, device, test_loader, evaluator, min_y, max_y)
train_perf, _, _ = eval(model, device, test_loader, evaluator)

/home/hy24/.conda/envs/ANPX/lib/python3.8/site-packages/torch_geometric/data/dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch.load(f)

This will take some time.......


/tmp/ipykernel_1689715/1439499710.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_name)['model_state_dict'])
Iteration: 100%|████

mape:[0.34566903]
mse:[13.864375]
smape:[0.24829891]
r2:[0.90020126]
